<h1>
<center>Graph Kernel PCA + LDA Classification </center>
</h1>

<font size="3"> 
In this notebook, we train and evaluate an LDA classifier while we apply a graph-based kernel PCA.
<br>   
    
Our datasets are regression problems, nevertheless, our target value range (0-1) helped us to solve the problem as a classification task. Where we rounded the target values and managed them as classes. The main reason that we use regression datasets is described in the report.
    
In more detail:   
- We use the PropagationAttr model for computing kernels with graph data. 
- PropagationAttr return adjacency matrixes of shape (num_of_grpahs,num_of_grpahs) for train and test.   
- We use these adjacency matrixes as input for a PCA model with a precomputed kernel.   
- The output of the KPCA model is an array with shape (num_of_grpahs,n_components).
- We use PCA output in order to feed it an LDA classifier using target with the shape of (num_of_grpahs,num_of_nodes)
- We make several experiments using different hyper-parameters for both KPCA and LDA models. 
    
    Ps: We use ParkingViolation and Chickenpox datasets for our experiments   
</font>

## Generals

<font size="3"> 
Packages import and system configurations. 
</font>

In [ ]:
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.multioutput import MultiOutputClassifier
from grakel.kernels import PropagationAttr
from sklearn.model_selection import ParameterGrid
import pandas as pd
import pickle
from tqdm import tqdm
from datetime import datetime as dt
import tensorflow as tf
from sklearn import metrics
import matplotlib.pyplot as plt
import multiprocessing
import os

cores = multiprocessing.cpu_count()-2
project_path = os.getcwd()

<font size="3"> 
Datasets paths. 
</font>

In [ ]:
#Parking Violation
G_train_path_park = project_path + '/Data/ParkingViolationPrediction_PCA_LDA/G_Train.pkl'
G_test_path_park = project_path + '/Data/ParkingViolationPrediction_PCA_LDA/G_Test.pkl'
test_targets_path_park = project_path + '/Data/ParkingViolationPrediction_PCA_LDA/Init/Test_Targets.csv'
train_targets_path_park = project_path + '/Data/ParkingViolationPrediction_PCA_LDA/Init/Train_Targets.csv'
test_mask_path_park = project_path + '/Data/ParkingViolationPrediction_PCA_LDA/Init/Test_Mask.csv'
K_train_park = project_path +'/Data/ParkingViolationPrediction_PCA_LDA/K_train.npy'
K_test_park = project_path +'/Data/ParkingViolationPrediction_PCA_LDA/K_test.npy'

#Chickenpox
G_train_path_chic = project_path + '/Data/Chickenpox/G2_Train.pkl'
G_test_path_chic = project_path + '/Data/Chickenpox/G2_Test.pkl'
test_targets_path_chic = project_path + '/Data/Chickenpox/Init/Chickenpox_Test_targets.csv'
train_targets_path_chic = project_path + '/Data/Chickenpox/Init/Chickenpox_Train_targets.csv'
test_mask_path_chic = None
K_train_chickenpox = project_path +'/Data/Chickenpox/K_train_ch.npy'
K_test_chickenpox = project_path +'/Data/Chickenpox/K_test_ch.npy'

## Data Preprocessing 

<font size="3"> 
A function that takes as input the data paths and return the data.
</font>

In [ ]:
def data_load(G_train_path,G_test_path,test_targets_path,train_targets_path,use_test_mask,test_mask_path):
    with open(G_train_path, 'rb') as inp:
        G_train = pickle.load(inp)
    with open(G_test_path, 'rb') as inp:
        G_test = pickle.load(inp)
        
    y_train = pd.read_csv(train_targets_path,sep=',', index_col=0)
    y_test = pd.read_csv(test_targets_path,sep=',', index_col=0)
    if use_test_mask:
        test_mask = pd.read_csv(test_mask_path,index_col=0)
    else:
        test_mask = None
    return G_train,G_test,y_train,y_test,test_mask

<font size="3"> 
A function that takes the datasets and return a subset for each data accoriding the given data-sizes.
</font>

In [ ]:
def get_subset(G_train,G_test,y_train,y_test,use_test_mask,test_mask,train_size,test_size):
    G_train = G_train[0:train_size]
    G_test = G_test[0:test_size]
    y_train = y_train.iloc[:,:train_size]
    y_test = y_test.iloc[:,:test_size]
    if use_test_mask:
        test_mask = test_mask.iloc[:,:test_size]
    else:
        test_mask = None
    return G_train,G_test,y_train,y_test,test_mask

<font size="3"> 
A function that get subset of the data if the given variable is True and reshape the targets to the necessary shape.
</font>

In [ ]:
def data_preprocess(G_train,G_test,y_train,y_test,use_test_mask,test_mask,subset,train_size,test_size):
    if subset:
        G_train,G_test,y_train,y_test,test_mask = get_subset(G_train,G_test,y_train,y_test,use_test_mask,test_mask,train_size,test_size)

    y_train = np.array(y_train.T)
    y_test = np.array(y_test.T)
    if use_test_mask:
        test_mask = np.array(test_mask.T)
    else:
        test_mask = None
    return G_train,G_test,y_train,y_test,test_mask

## Main functionality

<font size="3"> 
A function that calculates the Mean Absolute Error (MAE) and Mean Squared Error (MSE) between predictions and actual targets for train and test sets. 
In case of Parking data, it uses a mask in order to calculate the errors only for the raw targets.
    <br>
    
Although we have transformed our problem as a classification task, we assign classes to it as integers and compute metrics that are applied to regression tasks.
</font>

In [ ]:
def calculate_metrics_on_actuals(eval_set,y_pred,y_test,use_test_mask,test_mask):
    
    if eval_set == 'test':
        pred = []
        actual = []
        if use_test_mask:
            for i in range(0,(len(y_pred))):
                for k in range (0,len(y_pred[0])):
                    if test_mask[i][k] == 1:
                        prd = (y_pred[i][k]/100)
                        pred.append(float(prd))
                        act = (y_test[i][k]/100)
                        actual.append(float(act))
        else:
            for i in range(0,(len(y_pred))):
                for k in range (0,len(y_pred[0])):
                    prd = (y_pred[i][k]/100)
                    pred.append(float(prd))
                    act = (y_test[i][k]/100)
                    actual.append(float(act))
    
    elif eval_set == 'train':
        pred = []
        actual = []
        for i in range(0,(len(y_pred))):
            for k in range (0,len(y_pred[0])):
                prd = (y_pred[i][k]/100)
                pred.append(float(prd))
                act = (y_test[i][k]/100)
                actual.append(float(act))   
        
        
    MAE = round(metrics.mean_absolute_error(actual, pred),5)
    print (f"The Mean Abslolute Error (MAE) that have been calculated for {eval_set} set is: {MAE}")
    MSE = round(metrics.mean_squared_error(actual, pred),5)
    print (f"The Mean Squared Error (MSE) that have been calculated for {eval_set} set is: {MSE}")
    return MAE,MSE,pred,actual

<font size="3"> 
A function that converts our target values to classes.

In more detail, it scales target column from range (0-1) to (1-100)  
</font>

In [ ]:
def values_to_classes(df):
    df = df.round(decimals=2)
    df = df * 100
    df = df.astype('int')
    unique_classes = np.unique(df)
    return df,unique_classes

<font size="3"> 
A function that applies kernel computation using PropagationAttr model.

Function fit and saves adjacency matrixes or loads them according to the given instruction
</font>

In [ ]:
def graph_kernel_computation(fit,G_train,G_test,K_train_path,K_test_path):
    if fit:
        graph_kernels = PropagationAttr(t_max=10,w=20,M='L2',n_jobs=cores)
        graph_kernels.fit(G_train)
        K_train = graph_kernels.transform(G_train)
        K_test = graph_kernels.transform(G_test)
        with open(K_train_path, 'wb') as f:
             np.save(f, K_train)
        with open(K_test_path, 'wb') as f:
             np.save(f, K_test)
    else:
        with open(K_train_path, 'rb') as f:
             K_train = np.load(f)
        with open(K_test_path, 'rb') as f:
             K_test = np.load(f)
        
    return K_train,K_test    

## Kenel PCA Evaluation

<font size="3"> 
A function that helps us to define the best 'n_components' value
    
In more detail:
- Fit a precomputed Kernel PCA model using the given 'n_components'.
- Use KPCA outputs in order to fit an MultiOutput LDA model (default hyper parameter)
- Calculate the necessary metrics in order to find the best n_components
</font>

In [ ]:
def evaluate_kpca(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,N,K_train_path,K_test_path):
    start = dt.now()
    K_train,K_test = graph_kernel_computation(False,G_train,G_test,K_train_path,K_test_path)
    transformer = KernelPCA(kernel='precomputed',n_components=N )
    X_train_transformed = transformer.fit_transform(K_train)
    X_test_transformed = transformer.transform(K_test)
    
    y_train,unique_classes_train = values_to_classes(y_train)
    y_test,unique_classes_test = values_to_classes(y_test)
    LDA = LinearDiscriminantAnalysis()
    LDA_Mclassifier = MultiOutputClassifier(LDA)
    
    LDA_Mclassifier.fit(X_train_transformed, y_train)
    running_secs = (dt.now() - start).seconds
    print (f"\nLDA Model with PCA (n_components={N}) have fitted succesfully in {(dt.now() - start).seconds} seconds")
    
    y_train_pred = LDA_Mclassifier.predict(X_train_transformed)
    train_MAE,train_MSE,train_pred,train_actual = calculate_metrics_on_actuals('train',y_train_pred,y_train,use_test_mask,test_mask)
    
    y_pred = LDA_Mclassifier.predict(X_test_transformed)
    MAE,MSE,pred,actual = calculate_metrics_on_actuals('test',y_pred,y_test,use_test_mask,test_mask)
    
    return MAE,MSE

<font size="3"> 
A function that feed the above pipeline with different values of 'n_components' while saves the evaluation results.
</font>

In [ ]:
def kpca_evaluation_results(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,
                           n_components,K_train_park,K_test_park):
    
    all_mae = []
    all_mse = []
    for i in range(0,len(n_components)):
        MAE,MSE = evaluate_kpca(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,n_components[i],
                               K_train_park,K_test_park)
        all_mae.append(MAE)
        all_mse.append(MSE)
    pca_results={'n_components':n_components,'MAE':all_mae,'MSE':all_mse}
    min_mae = min(pca_results['MAE'])
    min_index = pca_results['MAE'].index(min_mae)
    best_n_components = pca_results['n_components'][min_index]
    return pca_results,best_n_components

<font size="3"> 
A function that plots the results of experiments involving finding the best hyper-parameter of 'n_components'
</font>

In [ ]:
def plot_kpca(pca_results,data_name):
    fig, (ax1, ax2) = plt.subplots(2)
    fig.suptitle('N_components infulence on PCA')    
    ax1.plot(range(1, len(pca_results['n_components']) + 1), pca_results['MAE'],label='MAE', color="blue")
    ax2.plot(range(1, len(pca_results['n_components']) + 1), pca_results['MSE'],label='MSE', color="orange")
    
    ax1.set(ylabel='MAE')
    ax2.set(ylabel='MSE')
    plt.xlabel('n_components', fontsize=12)
    plt.savefig('Exports/n_componets_Graph_PCA_' + data_name + '.pdf')
    plt.show()

## LDA evaluation

<font size="3"> 
A function that helps us to define the best hyper-parameters of LDA model 
    
In more detail:
- Fit a precomputed Kernel PCA model using the best 'n_components' found above.
- Use KPCA outputs in order to fit an MultiOutput LDA model with the given param_grid
- Calculate the necessary metrics in order to find the best hyper-parameters
</font>

In [ ]:
def evaluate_lda(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,param_grid,
                 best_n_components,K_train_path,K_test_path):
    
    K_train,K_test = graph_kernel_computation(False,G_train,G_test,K_train_path,K_test_path)
    
    start = dt.now()
    transformer = KernelPCA(kernel='precomputed',n_components=best_n_components)
    X_train_transformed = transformer.fit_transform(K_train)
    X_test_transformed = transformer.transform(K_test)

    y_train,unique_classes_train = values_to_classes(y_train)
    y_test,unique_classes_test = values_to_classes(y_test)
    
    LDA = LinearDiscriminantAnalysis(**param_grid)
    LDA_Mclassifier = MultiOutputClassifier(LDA)
    LDA_Mclassifier.fit(X_train_transformed,y_train)
    running_secs = (dt.now() - start).seconds
    print (f"LDA Model have fitted succesfully in {running_secs} seconds")
    
    y_train_pred = LDA_Mclassifier.predict(X_train_transformed)
    train_MAE,train_MSE,train_pred,train_actual = calculate_metrics_on_actuals('train',y_train_pred,y_train,use_test_mask,test_mask)
    
    y_pred = LDA_Mclassifier.predict(X_test_transformed)
    MAE,MSE,pred,actual = calculate_metrics_on_actuals('test',y_pred,y_test,use_test_mask,test_mask)
    
    return MAE,MSE,running_secs

<font size="3"> 
A function that feed the above pipeline with different LDA hyper-parameters while saves the evaluation results.
</font>

In [ ]:
def lda_evaluation_results(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,param_grid,
                                     best_n_components,K_train_path,K_test_path):
    
    lda_results = []
    for i in range(0,len(param_grid)):
        print (f"\nParameters: {list(param_grid)[i]}")
        MAE,MSE,running_secs = evaluate_lda(G_train,G_test,y_train,y_test,use_test_mask,test_mask,cores,param_grid[i],
                               best_n_components,K_train_path,K_test_path)
        lda_results.append({'Parameters':param_grid[i],'MAE':MAE,'MSE':MSE,'Training Time':running_secs})
    lda_results = pd.DataFrame(lda_results)
    lda_results = lda_results.sort_values(by=['MAE'])
    lda_results = lda_results.reset_index()
    lda_results = lda_results.drop(['index'], axis=1)
    return lda_results

## 1. Functionality Combinations for Parking Data 

### 1.1 DATA

In [ ]:
G_train,G_test,y_train,y_test,test_mask = data_load(G_train_path_park,G_test_path_park,test_targets_path_park,
                                                    train_targets_path_park,True,test_mask_path_park)

G_train,G_test,y_train,y_test,test_mask = data_preprocess(G_train,G_test,y_train,y_test,True,test_mask,
                                                          False,1000,100)

### 1.2 KPCA Evaluation

In [ ]:
n_components_park = np.arange(1,14,1)
pca_results_park,best_n_components_park = kpca_evaluation_results(G_train,G_test,y_train,y_test,True,test_mask,cores,
                                 n_components_park,K_train_park,K_test_park)
plot_kpca(pca_results_park,'Parking')

### 1.3 LDA evalutation

In [ ]:
parameters_park = [{"solver":['svd'],'store_covariance':[False,True],'tol':[0.5,0.05]},
                {"solver":['lsqr','eigen'],'shrinkage':['auto',None,0.2,0.4]}]
param_grid_park = ParameterGrid(parameters_park)
lda_results_park = lda_evaluation_results(G_train,G_test,y_train,y_test,True,test_mask,cores,param_grid_park,
                                     best_n_components_park,K_train_park,K_test_park)

In [ ]:
lda_results_park

## 2. Functionality Combinations for ChickenPox Data

### 2.1 DATA

In [ ]:
G_train,G_test,y_train,y_test,test_mask = data_load(G_train_path_chic,G_test_path_chic,test_targets_path_chic,
                                                    train_targets_path_chic,False,test_mask_path_chic)
G_train,G_test,y_train,y_test,test_mask = data_preprocess(G_train,G_test,y_train,y_test,False,test_mask,
                                                          False,250,60)

### 2.2 KPCA Evaluation

In [ ]:
n_components_chi = np.arange(1,5,1)
pca_results_chi,best_n_components_chi = kpca_evaluation_results(G_train,G_test,y_train,y_test,False,test_mask,cores,
                                 n_components_chi,K_train_chickenpox,K_test_chickenpox)
plot_kpca(pca_results_chi,'Chickenpox')

### 1.3 LDA evalutation

In [ ]:
parameters_chi = [{"solver":['svd'],'store_covariance':[False,True],'tol':[0.5,0.05]},
                {"solver":['lsqr','eigen'],'shrinkage':['auto',None,0.2,0.4]}]
param_grid_chi = ParameterGrid(parameters_chi)
lda_results_chi = lda_evaluation_results(G_train,G_test,y_train,y_test,False,test_mask,cores,param_grid_chi,
                                     best_n_components_chi,K_train_chickenpox,K_test_chickenpox)

In [ ]:
lda_results_chi